In [2]:
import requests
import pandas as pd
import pyodbc 
from datetime import date
import time
import uuid

In [3]:
url = 'https://data.exia.com.co/demo/data/last'
response = requests.get(url).json()
response

{'co2factor': 6571910.0,
 'created_at': 'May 22 2022 15:00:00',
 'day_energy': 45131.0,
 'id': 16086,
 'pac_sum': 6840.0,
 'pac_sum_counter': 0,
 'pac_sum_temp': 0.0,
 'power_counter': 1,
 'power_real': 39.7958,
 'power_real_temp': 53.3646,
 'reference': 'comfenalco1',
 'total_energy': 39980000.0,
 'update_at': 'May 22 2022 15:40:23'}

In [7]:
today = str(date.today())
def colecting_data(df):
    url = 'https://data.exia.com.co/demo/data/last'
    try:
        response = requests.get(url).json()
        time.sleep(1)
    except Exception as e:
        print(404)
    else:
        print(200)
    
    data_id = str(uuid.uuid1(clock_seq=3))
    co2factor = response["co2factor"]
    createddata = response["created_at"].split(" ")
    created_at = "{}".format(today)
    created_at_time = createddata[3] 
    day_energy = response["day_energy"]
    id = response["id"]
    pac_sum = response["pac_sum"]
    pac_sum_counter = response["pac_sum_counter"]
    pac_sum_temp = response["pac_sum_temp"]
    power_counter = response["power_counter"]
    power_real = response["power_real"]
    power_real_temp = response["power_real_temp"]
    reference = response["reference"]
    total_energy = response["total_energy"]
    updatedata = response["update_at"].split(" ")
    update_at = "{}".format(today)
    update_at_time = updatedata[3]

    df = df.append({"data_id":data_id,"co2factor": co2factor,"created_at" : created_at,"created_at_time":created_at_time,"day_energy" : day_energy,
    "id" : id,"pac_sum" : pac_sum,"pac_sum_counter" : pac_sum_counter,"pac_sum_temp": pac_sum_temp,
    "power_counter": power_counter,"power_real": power_real,"power_real_temp": power_real_temp,"reference": reference,
    "total_energy":total_energy,"update_at": update_at, "update_at_time":update_at_time}, ignore_index = True)

    return df
    

       

In [5]:
df = pd.DataFrame(columns=[
    'data_id',
    'co2factor',
    'created_at',
    "created_at_time",
    'day_energy',
    'id',
    'pac_sum',
    'pac_sum_counter',
    'pac_sum_temp',
    'power_counter',
    'power_real',
    'power_real_temp',
    'reference',
    'total_energy',
    'update_at',
    'update_at_time'
    ])

In [8]:
df = colecting_data(df)

200


C:\Users\Alberto\AppData\Local\Temp\ipykernel_12064\2721878795.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"data_id":data_id,"co2factor": co2factor,"created_at" : created_at,"created_at_time":created_at_time,"day_energy" : day_energy,


In [9]:
df.head()

,data_id,co2factor,created_at,created_at_time,day_energy,id,pac_sum,pac_sum_counter,pac_sum_temp,power_counter,power_real,power_real_temp,reference,total_energy,update_at,update_at_time
0,93634fb1-da0f-11ec-8003-544810b21542,6571930.0,2022-05-22,15:00:00,45259.0,16086,6888.0,0,0.0,1,19.6832,53.3646,comfenalco1,39980100.0,2022-05-22,15:41:32


In [11]:
#opening azure database
import mysql.connector as conn
try:
    db = conn.connect(
        user="casab001", 
        password="CA765@19", 
        host="casab.mysql.database.azure.com", 
        port=3306, 
        database="casabapidata"
    )  
    print("\n"*2)
    print("conexión exitosa", db)
    
except Exception as e:
    print("Ocurrió un error al conectar a SQL Server: ", e)





conexión exitosa <mysql.connector.connection_cext.CMySQLConnection object at 0x000001E190F371F0>


In [14]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS lastdata (
        data_id VARCHAR(255) PRIMARY KEY,
        co2factor FLOAT NOT NULL,
        created_at_date DATE NOT NULL,
        created_at_time TIME NOT NULL,
        day_energy FLOAT NOT NULL,
        id INTEGER NOT NULL,
        pac_sum FLOAT NOT NULL,
        pac_sum_counter INTEGER NOT NULL,
        pac_sum_temp FLOAT NOT NULL,
        power_counter INTEGER NOT NULL,	
        power_real FLOAT NOT NULL,	
        power_real_temp FLOAT NOT NULL,	
        reference VARCHAR(255) NOT NULL,	
        total_energy FLOAT NOT NULL,
        update_at_date DATE NOT NULL,
        update_at_time TIME NOT NULL
    )""")
    curr.execute(create_table_command)

In [12]:
cursor = db.cursor()

In [15]:
create_table(cursor)
db.commit()

In [93]:
#populating the database
insert_into_lastdata = ("""INSERT INTO lastdata (data_id,co2factor,created_at,day_energy,id,pac_sum,pac_sum_counter,pac_sum_temp,power_counter,power_real,power_real_temp,reference,total_energy,update_at)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
#row_to_insert = (,co2factor,created_at,day_energy,id,pac_sum,pac_sum_counter,pac_sum_temp,power_counter,power_real,power_real_temp,reference,total_energy,update_at)
df.data_id.to_list()


['b49fb285-d9f8-11ec-8003-544810b21542',
 '1dc8f235-d9f9-11ec-8003-544810b21542',
 '1f86f6d4-d9f9-11ec-8003-544810b21542']